
# **Proyecto Transmilenio para buscar la mejor ruta entre multiples estaciones usando algoritmos de busqueda de Inteligencia Artificial**

Hecho por: Carlos Andres Garnica Salazar - 625762
Universidad Catolica de Colombia

Este proyecto consiste en buscar la mejor ruta dentro del sistema de portales y estaciones de transmilenio. Para lograr esto se implementa un sistema de arboles y grafos para hacer busquedas no informadas e informadas con los algoritmos BSF, DFS, UCS y A*


---



Para hacer la representacion sobre un grafo en computacion necesitamos construir varias clases que representen a los nodos, los pesos y la conexion entre ellos.

La principal ventaja es que un arbol es un tipo de grafo, por lo tanto nos serviran las mismas clases para representarlo.

Ademas de las clases se utilizo diccionarios, que es la manera mas facil de representar este tipo de estructuras de datos en Python. Claro, a este tipo de diccionarios van a guardar objetos de instancias de clases (Vertex) con el fin de almacenar multiples valores y datos en un mismo objeto, en vez de hacer multiples diccionarios, accion de programacion que es menos viable.


**Mapa de estaciones**

![Mapa de estaciones del sistema de transporte Transmilenio.](https://www.sitp.gov.co/publicaciones/40236/mapas_transmilenio/info/sitp/media/img75080.jpg)

**El nombre de las estaciones pueden cambiar*



**Estaciones**

Las estaciones que se implementaron son las siguientes:

> *   Portal Norte
*   Toberin
*   Calle 161
*   Mazuren
*   Calle 146
*   Calle 142
*   Alcala
*   Prado
*   Calle 127
*   Pepe Sierra
*   Calle 106
*   Calle 100
*   La Castellana
*   NQS-Calle 75
*   AV. Chile
*   Simon Bolivar
*   Movistar Arena
*   Campin - U. Antonio Nariño
*   AV. El Dorado
*   CAD
*   Paloquemao
*   Ricaurte
*   San Façon
*   De La Sabana
*   AV. Jiménez
*   Virrey
*   Calle 85
*   Héroes
*   Calle 76
*   Calle 72
*   Flores
*   Calle 63
*   Calle 57
*   Marly
*   Calle 45
*   AV. 39
*   Calle 34
*   Calle 26
*   Calle 22
*   Calle 19
*   Concejo de Bogotá
*   Centro Memoria
*   U. Nacional

Como inicio de la ruta se escogio el Portal norte, mientras que el final de la ruta es la estacion AV. Jimenez.

# **Dependencias**

Este proyecto requiere de ciertas dependencias, las cuales se describen a continuacion:

*   `PriorityQueue`: Clase que permite implementar una cola de prioridad en Python. Para saber mas la documentacion esta [aqui](https://docs.python.org/3/library/queue.html).
*   `Geopy.distance`: Permite implementar metricas de distancia entre 2 puntos terrestes con coordenadas de latitud y longitud. En este caso el algoritmo a utilizar es great_circle. La documentacion de las distancias se encuentra en [este enlace](https://geopy.readthedocs.io/en/stable/#module-geopy.distance).



In [0]:
from queue import PriorityQueue
from geopy.distance import geodesic, great_circle

# **Clase Vertex**

> Para poder representar cada nodo del grafo o arbol, necesitamos una clase que englobe esto, ademas que guarde los vecinos o una lista de adyacentes por cada nodo. La siguiente clase se encargara de esto.

In [0]:
class Vertex:
    def __init__(self, node, coordinates):
        self.id = node
        self.coordinates = coordinates # (x,y)
        self.adjacent = {}

    def __str__(self):
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adjacent])

    def add_neighbor(self, neighbor, weight=0):
        self.adjacent[neighbor] = weight

    def get_connections(self):
        return self.adjacent.keys()  

    def get_childrens(self):
        return [x.id for x in self.adjacent]

    def get_vertex_childrens(self):
        return [x for x in self.adjacent]

    def get_id(self):
        return self.id

    def get_weight(self, neighbor):
        return self.adjacent[neighbor]

    def get_coordinates(self):
        return self.coordinates

    def get_dict_childrens_weights(self):
        dictionary = {}

        for node in self.adjacent:
            node_id = node.get_id()
            node_weight = self.get_weight(node)
            dictionary.setdefault(node_id, node_weight)

        return dictionary


    def __repr__(self):
        return '{}, {}'.format(repr(self.id), repr(self.coordinates))
    
    def __lt__(self,other):
        """
        Perform the less than operation (self < other).
        
        Args:
            other: the other Node to compare to
        """
        return (self.id < other.id )
    
    def __gt__(self,other):
        """
        Perform the greater than operation (self > other).
        
        Args:
            other: the other Node to compare to
        """
        return (self.id  > other.id )

    

# **Clase Grafo**

> Con esta clase se engloba la estructura de datos de un grafo para su correspondiente representacion. Ademas, algunos metodos de esta clase estan diseñados para crear vertices, obtener vertices u obtener el grafo en forma de diccionario de la forma `{int : [list]...}`

Para la construccion del grafo se utilizo diccionarios de la clase vertex.

In [0]:
class Graph:
    def __init__(self):
        self.vert_dict = {}
        self.num_vertices = 0

    def __iter__(self):
        return iter(self.vert_dict.values())

    def add_vertex(self, node, coordinates):
        self.num_vertices = self.num_vertices + 1
        new_vertex = Vertex(node, coordinates)
        self.vert_dict[node] = new_vertex
        return new_vertex

    def get_vertex(self, n):
        if n in self.vert_dict:
            return self.vert_dict[n]
        else:
            return None

    def add_edge(self, frm, to, cost = 0):
        if frm not in self.vert_dict:
            self.add_vertex(frm)
        if to not in self.vert_dict:
            self.add_vertex(to)

        self.vert_dict[frm].add_neighbor(self.vert_dict[to], cost)
        self.vert_dict[to].add_neighbor(self.vert_dict[frm], cost)

    def get_vertices(self):
        return self.vert_dict.keys()

    def get_dictionary(self):
        graph = {}
        for v in self.vert_dict.values():
            graph.setdefault(v.get_id(), v.get_childrens())

        return graph

    # Create an undirected graph by adding symmetric edges
    def make_undirected(self):
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.graph_dict.setdefault(b, {})[a] = dist

# **Clase Node**

> El uso de esta clase se limita a facilitar la programacion y el manejo de los vertices en el algoritmo A*. Si bien se pudo utilizar la clase Vertex para este objetivo, no es lo mas adecuado, puestoq ue tienen objetivos diferentes, ademas de parametros, algunos similares pero en su mayoria no implmenetados en Vertex por convencion de abstraccion del problema.

In [0]:
class Node:

    # Initialize the class
    def __init__(self, name, parent):
        self.name = name
        self.parent = parent
        self.g = 0 # Distance to start node
        self.h = 0 # Distance to goal node
        self.f = 0 # Total cost

    # Compare nodes
    def __eq__(self, other):
        return self.name == other.name

    # Sort nodes
    def __lt__(self, other):
         return self.f < other.f

    # Print node
    def __repr__(self):
        return ('({0},{1})'.format(self.position, self.f))

# **Clase TreeSearch**

En esta clase se dan todos los metodos de busqueda y recorrido del grafo, incluyendo la busqueda no informada e informada.

> *   BFS `(bfs_shortest_path)`: Busqueda por anchura. Este tipo de busqueda utiliza una cola para guardar los vecinos de un nodo.
*   DFS `(dfs_paths)`: Busqueda por profundidad. Este algoritmo utiliza una pila para almacenar todos los nodos adyacentes de los nodos que ha visitado
*   UCS `(ucs)`: Busqueda por costo unitario. Algoritmo similar a bfs, pero evalua los costos a traves de una cola de prioridades.
*   A* `(astar_search)`: Algoritmod e busqueda informada, el cual tiene en cuenta el costo unitario de un vertice con sus adyacentes, ademas de un costo heristico calculado

Para el calculo heuristico `(construct_heristic)` se midio como la distancia en linea recta desde la estacion AV. Jimenez a las demas estaciones (Distancia total). Esta medicion se hizo utilizando la libreria `Geopy.distance` en la funcion `Station.distance`, usando el algoritmo del gran circulo, el cual toma en cuenta que la tierra es totalmente esferica, con un radio de 6371.008 Km y dando resultados con un error del 0.5% aproximadamente.

Aun asi, se puede utilizar cualquier medicion tomando las coordenadas (longitud y latitud) por cada estacion, como la euclidiana o la distancia de manhattan para dicho proposito. El principal problema es que este tipo de distancias no tienen en cuenta la curvatura del arco, haciendola mas ineficiente para distancias en lugares de la tierra. Para el anterior trabajo esta el metodo `heristic`, el cual, dado 2 coordenadas, devuel ve las mediciones deseadas. Esta funcion se puede usar como complemento para medir todas las distancias, reemplazando a el algoritmo del gran circulo implementado en `GeoPy`.

> Si desea utilizar otro algoritmo de medicion de distancia, consulte la [documentacion de GeoPy para localizaciones](https://geopy.readthedocs.io/en/stable/#module-geopy.distance).





In [0]:
class TreeSearch:

    def __init__(self, graph):
        if graph is not None:
            self.graph = graph
            self.weights = []
        else:
            raise TypeError('El grafo tiene un valor de instancia incorrecto: None')

    def bfs_shortest_path(self, start, goal):
        if (isinstance(self.graph, dict)):
            raise TypeError('El grafo no es instancia de la clase Graph')

        explored = []

        queue = [[start]]
    
        if start == goal:
            return "El objetivo es el mismo que el inicio."
    
        while queue:
            
            path = queue.pop(0)
            
            node = path[-1]
            if node not in explored:
                neighbours = self.graph.vert_dict[node].get_childrens()
                
                for neighbour in neighbours:
                    new_path = list(path)
                    new_path.append(neighbour)
                    queue.append(new_path)
                    
                    if neighbour == goal:
                        return new_path
    
                explored.append(node)
    
        return "No hay conexion entre el nodo de inicio y el objetivo."

    def dfs_paths(self, start, goal):
        if(isinstance(self.graph, Graph)):
            raise TypeError('El grafo no es instancia de un diccionario.')

        stack = [(start, [start])]
        while stack:
            (vertex, path) = stack.pop()
            for next in set(self.graph[vertex]) - set(path):
                if next == goal:
                    return path + [next]
                else:
                    stack.append((next, path + [next]))

    def ucs(self, start, goal):
        if (isinstance(self.graph, dict)):
            raise TypeError('El grafo no es instancia de la clase Graph')

        visited = set()
        queue = PriorityQueue()
        queue.put((0, [start]))

        while queue:
            #cost, node = queue.get()
            pair = queue.get()
            current = pair[1][-1]
            #print(pair[1], type(pair[1]))
            if current not in visited:
                visited.add(current)
                
                
                if current == goal:
                    path = []
                    for u in pair[1]:
                        path.append(u.get_id())
                    return path
                
                for i in self.graph.vert_dict[current.get_id()].get_connections():
                    if i not in visited:
                        total_cost = pair[0] + self.graph.vert_dict[current.get_id()].get_weight(i)
                        new_path = list(pair[1])
                        new_path.append(i)
                        queue.put((total_cost, new_path))
    
    def heuristic(self, graph, a, b, method = 'euclidean'):
        #GET WEIGHTS
        node_a = graph.get_vertex(a)
        node_b = graph.get_vertex(b)

        (x1, y1) = node_a.get_coordinates()
        (x2, y2) = node_b.get_coordinates()
        if method == 'euclidean':
            return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5
        elif method == 'manhattan':
            return abs(x1 - x2) + abs(y1 - y2)

    def astar_search(self, graph, heuristics, start, end):
        if (isinstance(self.graph, dict)):
            raise TypeError('El grafo no es instancia de la clase Graph')
        # Create lists for open nodes and closed nodes
        open = []
        closed = []

        # Create a start node and an goal node
        start_node = Node(start, None)
        goal_node = Node(end, None)

        # Add the start node
        open.append(start_node)
        
        # Loop until the open list is empty
        while len(open) > 0:

            # Sort the open list to get the node with the lowest cost first
            open.sort()

            # Get the node with the lowest cost
            current_node = open.pop(0)

            # Add the current node to the closed list
            closed.append(current_node)
            
            # Check if we have reached the goal, return the path
            if current_node == goal_node:
                path = []
                while current_node != start_node:
                    path.append(str(current_node.name) + ': ' + str(current_node.g))
                    current_node = current_node.parent
                path.append(str(start_node.name) + ': ' + str(start_node.g))
                # Return reversed path
                return path[::-1]

            # Get neighbours
            
            #neighbors = graph.get()
            neighbors = graph.get_vertex(current_node.name).get_dict_childrens_weights()

            # Loop neighbors
            for key, value in neighbors.items():

                # Create a neighbor node
                neighbor = Node(key, current_node)

                # Check if the neighbor is in the closed list
                if(neighbor in closed):
                    continue

                # Calculate full path cost
                neig_node = graph.get_vertex(neighbor.name)
                cost = graph.get_vertex(current_node.name).get_weight(neig_node)
                neighbor.g = current_node.g + cost
                neighbor.h = heuristics.get(neighbor.name)
                neighbor.f = neighbor.g + neighbor.h

                # Check if neighbor is in open list and if it has a lower f value
                if(self.add_to_open(open, neighbor) == True):
                    # Everything is green, add neighbor to open list
                    open.append(neighbor)

        # Return None, no path is found
        return None

    # Check if a neighbor should be added to open list
    def add_to_open(self, open, neighbor):
        for node in open:
            if (neighbor == node and neighbor.f > node.f):
                return False
        return True

    def construct_heristic(self, graph, station):
        heuristic = {}
        frm = graph.get_vertex(24)
        coordinates_from = frm.get_coordinates()
        for i in range(graph.num_vertices):
            to_node = graph.get_vertex(i)
            coordinates_to = to_node.get_coordinates()
            dist = station.distance(coordinates_from, coordinates_to)
            heuristic.setdefault(i, dist)

        return heuristic

# **Clase Station**
Esta clase se encarga de almacenar el diccionario completo de estaciones que se van a utilizar. Para la construccion de este tipo de estructura de datos se utiliza la funcion `build_all_stations`, la cual agrega el nombre de la estacion, y una lista que contiene el id que va a tener dentro de la ejecucion de este proyecto en conjunto con las coordenadas terrestres (latitud, longitud) en la que se ubica la estacion.

> Si bien, este tipo de coordenadas se pueden obtener mediante la libreria `GeoPy` y utilizando algun API de un mapa online como Google maps o Here Maps, requiere realizar la busqueda de la ubicacion exacta para que de como resultado las coordenadas. El problema es que alguna de estas APIs arrojan coordenadas aproximadas o incorrectas de la ubicacion a consultar, sabiendo que la consulta o la busqueda fue correcta.

Es por ello que se ha ingresado todas las coordenadas a mano, extrayendolas del sitio de Google Maps una a una. Es mas tedioso pero asegura que las coordenadas sean las mas exactas posibles.

Aunque hay algunos mapas libres, no dan las coordenadas exactas, y otros requieren una llave de la API *(API key)* para funcionar, y para que esta tenga alguna utilidad el desarrollador se ve obligado a pagar, como es el caso de Google Maps.

In [0]:
class Station:

    def __init__(self):
        self.stations = {}
    
    def build_all_stations(self):

        self.stations.setdefault('Portal Norte', [0, (4.754228, -74.046161)])
        self.stations.setdefault('Toberin', [1, (4.746185, -74.047279)])
        self.stations.setdefault('Calle 161', [2, (4.742706, -74.047863)])
        self.stations.setdefault('Mazuren', [3, (4.734499, -74.049242)]) 
        self.stations.setdefault('Calle 146', [4, (4.730832, -74.049868)]) 
        self.stations.setdefault('Calle 142', [5, (4.726947, -74.050305)]) 
        self.stations.setdefault('Alcala', [6, (4.720287, -74.051641)])
        self.stations.setdefault('Prado', [7, (4.713173, -74.052682)])
        self.stations.setdefault('Calle 127', [8, (4.704787, -74.054230)])
        self.stations.setdefault('Pepe Sierra', [9, (4.698795, -74.055251)])
        self.stations.setdefault('Calle 106', [10, (4.691557, -74.056421)])
        self.stations.setdefault('Calle 100', [11, (4.684800, -74.057570)])
        self.stations.setdefault('La Castellana', [12, (4.676243, -74.063387)])
        self.stations.setdefault('NQS-Calle 75', [13, (4.670653, -74.070593)])
        self.stations.setdefault('AV. Chile', [14, (4.665962, -74.074819)])
        self.stations.setdefault('Simon Bolivar', [15, (4.658008, -74.077797)])
        self.stations.setdefault('Movistar Arena', [16, (4.650119, -74.078363)])
        self.stations.setdefault('Campin - U. Antonio Nariño', [17, (4.644847, -74.078777)])
        self.stations.setdefault('AV. El Dorado', [18, (4.630541, -74.079891)])
        self.stations.setdefault('CAD', [19, (4.622983, -74.084559)])
        self.stations.setdefault('Paloquemao', [20, (4.617084, -74.089525)])
        self.stations.setdefault('Ricaurte', [21, (4.612523, -74.093075)])
        self.stations.setdefault('San Façon', [22, (4.609549, -74.086540)])
        self.stations.setdefault('De La Sabana', [23, (4.605659, -74.082138)])
        self.stations.setdefault('AV. Jiménez', [24, (4.603037, -74.079164)])
        self.stations.setdefault('Virrey', [25, (4.675857, -74.059144)])
        self.stations.setdefault('Calle 85', [26, (4.671851, -74.059702)])
        self.stations.setdefault('Héroes', [27, (4.668311, -74.060210)])
        self.stations.setdefault('Calle 76', [28, (4.664031, -74.061083)])
        self.stations.setdefault('Calle 72', [29, (4.659261, -74.061922)])
        self.stations.setdefault('Flores', [30, (4.654878, -74.063021)])
        self.stations.setdefault('Calle 63', [31, (4.648914, -74.064810)])
        self.stations.setdefault('Calle 57', [32, (4.642917, -74.065879)])
        self.stations.setdefault('Marly', [33, (4.636587, -74.066936)])
        self.stations.setdefault('Calle 45', [34, (4.632661, -74.067665)])
        self.stations.setdefault('AV. 39', [35, (4.627184, -74.068643)])
        self.stations.setdefault('Calle 34', [36, (4.621390, -74.069805)])
        self.stations.setdefault('Calle 26', [37, (4.616961, -74.072159)])
        self.stations.setdefault('Calle 22', [38, (4.611033, -74.075079)])
        self.stations.setdefault('Calle 19', [39, (4.608302, -74.076608)])
        self.stations.setdefault('Concejo de Bogotá', [40, (4.626496, -74.080722)])
        self.stations.setdefault('Centro Memoria', [41, (4.621915, -74.077436)])
        self.stations.setdefault('U. Nacional', [42, (4.636493, -74.079328)])
    
    def distance(self, frm, to):
        #print(frm, to)
        return float(great_circle(frm, to).meters)

    #CREAR CONEXIONES-ARISTAS
    def set_connections_stations(self, frm, to):
        frm = vertex[self.stations[frm][0]]
        to = vertex[self.stations[to][0]]

        cost = self.distance(frm.get_coordinates(), to.get_coordinates())
        g.add_edge(frm.get_id(), to.get_id(), cost)

    def get_station_by_id(self, dictionary, value):
        for s in dictionary:
            if dictionary[s][0] == value:
                return s
        return None

    def convert_id_to_station(self, list_stations_id):
        new_list = []
        for id in list_stations_id:
            name_station = self.get_station_by_id(self.stations, id)
            new_list.append(name_station)
        return new_list

# **Inicializacion de objetos**

Para poder utilizar las funciones, crear y tener acceso al grafo y a las estaciones se procede a instanciar objetos de algunas clases vistas anteriormente.

> `g` sera el objeto del grafo, almacena el grafo y los vertices, ademas tiene las operaciones para agregar vertices y aristas con los pesos correspondientes.

> `s` va a ser el objeto para la clase `Station`, el cual contiene el diccionario con las estaciones a crear y las operaciones sobre estas estaciones.

Por ultimo, tenemos la variable `tree`, el cual actuara como instancia de la clase `TreeSearch`, y contendra todos los elementos para la busqueda de rutas mas cortas utilizando diferentes algoritmos informados y no informados.

In [0]:
g = Graph()
s = Station()
tree = TreeSearch(g)

# **Construccion de las estaciones**

Mediante el metodo `build_all_stations` de la clase `Station` construimos todas las estaciones que vamos a utilizar y posteriormente guardandolas en el diccionario que se aloja en la anterior clase mencionada.

Por ultimo verificamos que todas las estaciones se hayan ingresado correctamente.

In [8]:
s.build_all_stations()
s.stations

{'AV. 39': [35, (4.627184, -74.068643)],
 'AV. Chile': [14, (4.665962, -74.074819)],
 'AV. El Dorado': [18, (4.630541, -74.079891)],
 'AV. Jiménez': [24, (4.603037, -74.079164)],
 'Alcala': [6, (4.720287, -74.051641)],
 'CAD': [19, (4.622983, -74.084559)],
 'Calle 100': [11, (4.6848, -74.05757)],
 'Calle 106': [10, (4.691557, -74.056421)],
 'Calle 127': [8, (4.704787, -74.05423)],
 'Calle 142': [5, (4.726947, -74.050305)],
 'Calle 146': [4, (4.730832, -74.049868)],
 'Calle 161': [2, (4.742706, -74.047863)],
 'Calle 19': [39, (4.608302, -74.076608)],
 'Calle 22': [38, (4.611033, -74.075079)],
 'Calle 26': [37, (4.616961, -74.072159)],
 'Calle 34': [36, (4.62139, -74.069805)],
 'Calle 45': [34, (4.632661, -74.067665)],
 'Calle 57': [32, (4.642917, -74.065879)],
 'Calle 63': [31, (4.648914, -74.06481)],
 'Calle 72': [29, (4.659261, -74.061922)],
 'Calle 76': [28, (4.664031, -74.061083)],
 'Calle 85': [26, (4.671851, -74.059702)],
 'Campin - U. Antonio Nariño': [17, (4.644847, -74.078777)]

# **Agregar vertices al grafo**

Cada estacion es un estado del problema, por lo tanto cada estacion sera un vertice del grafo.

> Para lograr esto se añadira cada estacion como un vertice, apoyandonos en la funcion `add_vertex` de la clase `Graph` utilizando solamente el id de la estacion y las coordenadas que estan guardadas en el diccionario de estaciones.

Por ultimo se verifican que los vertices se hayan agregado correctamente y que se haya creado el diccionario correctamente guardando todos los ids en un nuevo diccionario de muestra.

In [9]:
vertex = {}
#CREAR VERTICES
i = 0
for index in s.stations:
    v = g.add_vertex(s.stations[index][0], s.stations[index][1])
    #print(s.stations[index][0], s.stations[index][1])
    vertex.setdefault(i, v)
    i = i + 1

vertex

{0: 0, (4.754228, -74.046161),
 1: 1, (4.746185, -74.047279),
 2: 2, (4.742706, -74.047863),
 3: 3, (4.734499, -74.049242),
 4: 4, (4.730832, -74.049868),
 5: 5, (4.726947, -74.050305),
 6: 6, (4.720287, -74.051641),
 7: 7, (4.713173, -74.052682),
 8: 8, (4.704787, -74.05423),
 9: 9, (4.698795, -74.055251),
 10: 10, (4.691557, -74.056421),
 11: 11, (4.6848, -74.05757),
 12: 12, (4.676243, -74.063387),
 13: 13, (4.670653, -74.070593),
 14: 14, (4.665962, -74.074819),
 15: 15, (4.658008, -74.077797),
 16: 16, (4.650119, -74.078363),
 17: 17, (4.644847, -74.078777),
 18: 18, (4.630541, -74.079891),
 19: 19, (4.622983, -74.084559),
 20: 20, (4.617084, -74.089525),
 21: 21, (4.612523, -74.093075),
 22: 22, (4.609549, -74.08654),
 23: 23, (4.605659, -74.082138),
 24: 24, (4.603037, -74.079164),
 25: 25, (4.675857, -74.059144),
 26: 26, (4.671851, -74.059702),
 27: 27, (4.668311, -74.06021),
 28: 28, (4.664031, -74.061083),
 29: 29, (4.659261, -74.061922),
 30: 30, (4.654878, -74.063021),
 31

In [10]:
g.get_dictionary()

{0: [],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [],
 18: [],
 19: [],
 20: [],
 21: [],
 22: [],
 23: [],
 24: [],
 25: [],
 26: [],
 27: [],
 28: [],
 29: [],
 30: [],
 31: [],
 32: [],
 33: [],
 34: [],
 35: [],
 36: [],
 37: [],
 38: [],
 39: [],
 40: [],
 41: [],
 42: []}

# **Impresion de las estaciones**

Para verificar que las estaciones e hayan agregado se imprime el id y el nombre de la estacion.

In [11]:
for station in s.stations:
    print(station)

Portal Norte
Toberin
Calle 161
Mazuren
Calle 146
Calle 142
Alcala
Prado
Calle 127
Pepe Sierra
Calle 106
Calle 100
La Castellana
NQS-Calle 75
AV. Chile
Simon Bolivar
Movistar Arena
Campin - U. Antonio Nariño
AV. El Dorado
CAD
Paloquemao
Ricaurte
San Façon
De La Sabana
AV. Jiménez
Virrey
Calle 85
Héroes
Calle 76
Calle 72
Flores
Calle 63
Calle 57
Marly
Calle 45
AV. 39
Calle 34
Calle 26
Calle 22
Calle 19
Concejo de Bogotá
Centro Memoria
U. Nacional


In [12]:
for station in s.stations:
    print(s.stations[station][0], station)

0 Portal Norte
1 Toberin
2 Calle 161
3 Mazuren
4 Calle 146
5 Calle 142
6 Alcala
7 Prado
8 Calle 127
9 Pepe Sierra
10 Calle 106
11 Calle 100
12 La Castellana
13 NQS-Calle 75
14 AV. Chile
15 Simon Bolivar
16 Movistar Arena
17 Campin - U. Antonio Nariño
18 AV. El Dorado
19 CAD
20 Paloquemao
21 Ricaurte
22 San Façon
23 De La Sabana
24 AV. Jiménez
25 Virrey
26 Calle 85
27 Héroes
28 Calle 76
29 Calle 72
30 Flores
31 Calle 63
32 Calle 57
33 Marly
34 Calle 45
35 AV. 39
36 Calle 34
37 Calle 26
38 Calle 22
39 Calle 19
40 Concejo de Bogotá
41 Centro Memoria
42 U. Nacional


# **Conectar estaciones**

Utilizando la funcion `Station.distance` para obtener el costo entre 2 estaciones y luego `Graph.add_edge` para conectar 2 vertices, se procede a crear las conexiones entre 2 estaciones con el metodo `Station.set_connections_stations`, pasandole como parametros el nombre de las 2 estaciones a conectar. Este metodo busca las estaciones y escoge el id de cada estacion para luego buscar su vertice correspondiente con este numero de identificacion qeu se aloja en el diccionario del grafo, conectandolos asi utilizando el objeto `g` de la clase `Graph.`.

In [13]:
s.set_connections_stations('Portal Norte', 'Toberin')
s.set_connections_stations('Toberin', 'Calle 161')
s.set_connections_stations('Calle 161', 'Mazuren')
s.set_connections_stations('Mazuren', 'Calle 146')
s.set_connections_stations('Calle 146', 'Calle 142')
s.set_connections_stations('Calle 142', 'Alcala')
s.set_connections_stations('Alcala', 'Prado')
s.set_connections_stations('Prado', 'Calle 127')
s.set_connections_stations('Calle 127', 'Pepe Sierra')
s.set_connections_stations('Pepe Sierra', 'Calle 106')
s.set_connections_stations('Calle 106', 'Calle 100')
s.set_connections_stations('Calle 100', 'La Castellana')
s.set_connections_stations('La Castellana', 'NQS-Calle 75')
s.set_connections_stations('NQS-Calle 75', 'AV. Chile')
s.set_connections_stations('AV. Chile', 'Simon Bolivar')
s.set_connections_stations('Simon Bolivar', 'Movistar Arena')
s.set_connections_stations('Movistar Arena', 'Campin - U. Antonio Nariño')
s.set_connections_stations('Campin - U. Antonio Nariño', 'U. Nacional')
s.set_connections_stations('U. Nacional', 'AV. El Dorado')
s.set_connections_stations('AV. El Dorado', 'CAD')
s.set_connections_stations('CAD', 'Paloquemao')
s.set_connections_stations('Paloquemao', 'Ricaurte')
s.set_connections_stations('Ricaurte', 'San Façon')
s.set_connections_stations('San Façon', 'De La Sabana')
s.set_connections_stations('De La Sabana', 'AV. Jiménez')
s.set_connections_stations('Calle 100', 'Virrey')
s.set_connections_stations('Virrey', 'Calle 85')
s.set_connections_stations('Calle 85', 'Héroes')
s.set_connections_stations('Héroes', 'Calle 76')
s.set_connections_stations('Calle 76', 'Calle 72')
s.set_connections_stations('Calle 72', 'Flores')
s.set_connections_stations('Flores', 'Calle 63')
s.set_connections_stations('Calle 63', 'Calle 57')
s.set_connections_stations('Calle 57', 'Marly')
s.set_connections_stations('Marly', 'Calle 45')
s.set_connections_stations('Calle 45', 'AV. 39')
s.set_connections_stations('AV. 39', 'Calle 34')
s.set_connections_stations('Calle 34', 'Calle 26')
s.set_connections_stations('Calle 26', 'Calle 22')
s.set_connections_stations('Calle 22', 'Calle 19')
s.set_connections_stations('Calle 19', 'AV. Jiménez')
s.set_connections_stations('Calle 26', 'Centro Memoria')
s.set_connections_stations('Centro Memoria', 'Concejo de Bogotá')
s.set_connections_stations('Concejo de Bogotá', 'AV. El Dorado')
s.set_connections_stations('Concejo de Bogotá', 'CAD')

g.get_dictionary()

{0: [1],
 1: [0, 2],
 2: [1, 3],
 3: [2, 4],
 4: [3, 5],
 5: [4, 6],
 6: [5, 7],
 7: [6, 8],
 8: [7, 9],
 9: [8, 10],
 10: [9, 11],
 11: [10, 12, 25],
 12: [11, 13],
 13: [12, 14],
 14: [13, 15],
 15: [14, 16],
 16: [15, 17],
 17: [16, 42],
 18: [42, 19, 40],
 19: [18, 20, 40],
 20: [19, 21],
 21: [20, 22],
 22: [21, 23],
 23: [22, 24],
 24: [23, 39],
 25: [11, 26],
 26: [25, 27],
 27: [26, 28],
 28: [27, 29],
 29: [28, 30],
 30: [29, 31],
 31: [30, 32],
 32: [31, 33],
 33: [32, 34],
 34: [33, 35],
 35: [34, 36],
 36: [35, 37],
 37: [36, 38, 41],
 38: [37, 39],
 39: [38, 24],
 40: [41, 18, 19],
 41: [37, 40],
 42: [17, 18]}

**Verificar Conexion**

Se imprime la creacion del grafo, la conexion entre cada nodo y su peso. Por ultimo se muestran los vertices adyacentes por cada vertice.

In [14]:
for v in g:
    for w in v.get_connections():
        vid = v.get_id()
        wid = w.get_id()
        print ('( %s , %s, %3d)'  % ( vid, wid, v.get_weight(w)))

for v in g:
    print ('g.vert_dict[%s]=%s' %(v.get_id(), g.vert_dict[v.get_id()]))


( 0 , 1, 902)
( 1 , 0, 902)
( 1 , 2, 392)
( 2 , 1, 392)
( 2 , 3, 925)
( 3 , 2, 925)
( 3 , 4, 413)
( 4 , 3, 413)
( 4 , 5, 434)
( 5 , 4, 434)
( 5 , 6, 755)
( 6 , 5, 755)
( 6 , 7, 799)
( 7 , 6, 799)
( 7 , 8, 948)
( 8 , 7, 948)
( 8 , 9, 675)
( 9 , 8, 675)
( 9 , 10, 815)
( 10 , 9, 815)
( 10 , 11, 762)
( 11 , 10, 762)
( 11 , 12, 1149)
( 11 , 25, 1009)
( 12 , 11, 1149)
( 12 , 13, 1011)
( 13 , 12, 1011)
( 13 , 14, 701)
( 14 , 13, 701)
( 14 , 15, 944)
( 15 , 14, 944)
( 15 , 16, 879)
( 16 , 15, 879)
( 16 , 17, 588)
( 17 , 16, 588)
( 17 , 42, 930)
( 18 , 42, 664)
( 18 , 19, 986)
( 18 , 40, 459)
( 19 , 18, 986)
( 19 , 20, 856)
( 19 , 40, 577)
( 20 , 19, 856)
( 20 , 21, 641)
( 21 , 20, 641)
( 21 , 22, 796)
( 22 , 21, 796)
( 22 , 23, 652)
( 23 , 22, 652)
( 23 , 24, 440)
( 24 , 23, 440)
( 24 , 39, 650)
( 25 , 11, 1009)
( 25 , 26, 449)
( 26 , 25, 449)
( 26 , 27, 397)
( 27 , 26, 397)
( 27 , 28, 485)
( 28 , 27, 485)
( 28 , 29, 538)
( 29 , 28, 538)
( 29 , 30, 502)
( 30 , 29, 502)
( 30 , 31, 692)
( 31 , 3

In [15]:
#IMPRIMIMOS LOS NODOS ADYACENTES DEL VERTICE UNO CON SUS RESPECTIVOS PESOS
g.get_vertex(1).adjacent

{0, (4.754228, -74.046161): 902.8821779537834,
 2, (4.742706, -74.047863): 392.22343861011296}

In [16]:
#IMPRIMIR UN DICCIONARIO CON LOS NODOS ADYACENTES Y SUS PESOS DEL NODO 1
g.get_vertex(1).get_dict_childrens_weights()

{0: 902.8821779537834, 2: 392.22343861011296}

In [17]:
#IMPRIMIR EL DICCIONARIO DEL GRAFO COMPLETO
#TIENE EL FORMATO {id: obj<Graph>}
g.vert_dict

{0: 0, (4.754228, -74.046161),
 1: 1, (4.746185, -74.047279),
 2: 2, (4.742706, -74.047863),
 3: 3, (4.734499, -74.049242),
 4: 4, (4.730832, -74.049868),
 5: 5, (4.726947, -74.050305),
 6: 6, (4.720287, -74.051641),
 7: 7, (4.713173, -74.052682),
 8: 8, (4.704787, -74.05423),
 9: 9, (4.698795, -74.055251),
 10: 10, (4.691557, -74.056421),
 11: 11, (4.6848, -74.05757),
 12: 12, (4.676243, -74.063387),
 13: 13, (4.670653, -74.070593),
 14: 14, (4.665962, -74.074819),
 15: 15, (4.658008, -74.077797),
 16: 16, (4.650119, -74.078363),
 17: 17, (4.644847, -74.078777),
 18: 18, (4.630541, -74.079891),
 19: 19, (4.622983, -74.084559),
 20: 20, (4.617084, -74.089525),
 21: 21, (4.612523, -74.093075),
 22: 22, (4.609549, -74.08654),
 23: 23, (4.605659, -74.082138),
 24: 24, (4.603037, -74.079164),
 25: 25, (4.675857, -74.059144),
 26: 26, (4.671851, -74.059702),
 27: 27, (4.668311, -74.06021),
 28: 28, (4.664031, -74.061083),
 29: 29, (4.659261, -74.061922),
 30: 30, (4.654878, -74.063021),
 31

In [18]:
#DICCIONARIO RESUMIDO DEL GRAFO Y SUIS CONEXIONES
g.get_dictionary()

{0: [1],
 1: [0, 2],
 2: [1, 3],
 3: [2, 4],
 4: [3, 5],
 5: [4, 6],
 6: [5, 7],
 7: [6, 8],
 8: [7, 9],
 9: [8, 10],
 10: [9, 11],
 11: [10, 12, 25],
 12: [11, 13],
 13: [12, 14],
 14: [13, 15],
 15: [14, 16],
 16: [15, 17],
 17: [16, 42],
 18: [42, 19, 40],
 19: [18, 20, 40],
 20: [19, 21],
 21: [20, 22],
 22: [21, 23],
 23: [22, 24],
 24: [23, 39],
 25: [11, 26],
 26: [25, 27],
 27: [26, 28],
 28: [27, 29],
 29: [28, 30],
 30: [29, 31],
 31: [30, 32],
 32: [31, 33],
 33: [32, 34],
 34: [33, 35],
 35: [34, 36],
 36: [35, 37],
 37: [36, 38, 41],
 38: [37, 39],
 39: [38, 24],
 40: [41, 18, 19],
 41: [37, 40],
 42: [17, 18]}

In [19]:
#ESTACIONES CREADAS
s.stations

{'AV. 39': [35, (4.627184, -74.068643)],
 'AV. Chile': [14, (4.665962, -74.074819)],
 'AV. El Dorado': [18, (4.630541, -74.079891)],
 'AV. Jiménez': [24, (4.603037, -74.079164)],
 'Alcala': [6, (4.720287, -74.051641)],
 'CAD': [19, (4.622983, -74.084559)],
 'Calle 100': [11, (4.6848, -74.05757)],
 'Calle 106': [10, (4.691557, -74.056421)],
 'Calle 127': [8, (4.704787, -74.05423)],
 'Calle 142': [5, (4.726947, -74.050305)],
 'Calle 146': [4, (4.730832, -74.049868)],
 'Calle 161': [2, (4.742706, -74.047863)],
 'Calle 19': [39, (4.608302, -74.076608)],
 'Calle 22': [38, (4.611033, -74.075079)],
 'Calle 26': [37, (4.616961, -74.072159)],
 'Calle 34': [36, (4.62139, -74.069805)],
 'Calle 45': [34, (4.632661, -74.067665)],
 'Calle 57': [32, (4.642917, -74.065879)],
 'Calle 63': [31, (4.648914, -74.06481)],
 'Calle 72': [29, (4.659261, -74.061922)],
 'Calle 76': [28, (4.664031, -74.061083)],
 'Calle 85': [26, (4.671851, -74.059702)],
 'Campin - U. Antonio Nariño': [17, (4.644847, -74.078777)]

# **Escoger estaciones de inicio y destino**

Imaginemos que vamos a realizar un viaje desde una estacion a otra. para este caso se ha elegido como inicio el Portal Norte hasta la AV. Jimenez (Nodo 0 y nodo 24 correspondientemente).

Para lograr esto obtenemos los nodos con la funcion `Graph.get_vertex` y los guardamos en una variable, lo que nos devolvera sera un objeto de la clase `Vertex`.

In [0]:
node_from = g.get_vertex(0)
node_to = g.get_vertex(24)

# **Ejecucion de los algoritmos de busqueda**

Se procede a llamar a los metodos de busqueda alojados en la clase `TreeSearch`.

> Todos los metodos devuelven la ruta mas corta en forma de id de cada nodo. En el caso de A* devuelve un arreglo con la ruta optima y su costo acumulado. Para eliminar este costo se coge solamente la parte del id del nodo que corresponde la camino de la ruta mas optima decidida por este algoritmo.

In [0]:
bfs = tree.bfs_shortest_path(node_from.get_id(), node_to.get_id())
ucs = tree.ucs(node_from, node_to)

tree.graph = g.get_dictionary()
dfs = tree.dfs_paths(node_from.get_id(), node_to.get_id())

tree.graph = g

heuristic = tree.construct_heristic(g, s)
a_star = tree.astar_search(g, heuristic, node_from.get_id(), node_to.get_id())

for index, element in enumerate(a_star):
    cad = element.split(':')
    node = cad[0]
    a_star[index] = int(node)

#dijkstra = tree.a_star_search(g, node_from.get_id(), node_to.get_id(), heuris=False)


# **Conversion de id a nombre de la estacion**

Con el diccionario de estaciones podemos buscar que numero de identificacion tiene. Le pasamos la lista que contiene la ruta optima de cada algoritmo al metodo `Station.convert_id_to_station` para que nos devuelva la misma ruta pero con los nombres de la estacion.

In [0]:
bfs_convert = s.convert_id_to_station(bfs)
ucs_convert = s.convert_id_to_station(ucs)
dfs_convert = s.convert_id_to_station(dfs)
astar_convert = s.convert_id_to_station(a_star)


#print('Dijkstra: ', dijkstra)

# **Impresion de las rutas optimas por algoritmo**

In [23]:
print("BFS: " + str(bfs_convert))
print('UCS: ' + str(ucs_convert))
print("DFS: " + str(dfs_convert))
print('A*: ', astar_convert)

BFS: ['Portal Norte', 'Toberin', 'Calle 161', 'Mazuren', 'Calle 146', 'Calle 142', 'Alcala', 'Prado', 'Calle 127', 'Pepe Sierra', 'Calle 106', 'Calle 100', 'La Castellana', 'NQS-Calle 75', 'AV. Chile', 'Simon Bolivar', 'Movistar Arena', 'Campin - U. Antonio Nariño', 'U. Nacional', 'AV. El Dorado', 'CAD', 'Paloquemao', 'Ricaurte', 'San Façon', 'De La Sabana', 'AV. Jiménez']
UCS: ['Portal Norte', 'Toberin', 'Calle 161', 'Mazuren', 'Calle 146', 'Calle 142', 'Alcala', 'Prado', 'Calle 127', 'Pepe Sierra', 'Calle 106', 'Calle 100', 'Virrey', 'Calle 85', 'Héroes', 'Calle 76', 'Calle 72', 'Flores', 'Calle 63', 'Calle 57', 'Marly', 'Calle 45', 'AV. 39', 'Calle 34', 'Calle 26', 'Calle 22', 'Calle 19', 'AV. Jiménez']
DFS: ['Portal Norte', 'Toberin', 'Calle 161', 'Mazuren', 'Calle 146', 'Calle 142', 'Alcala', 'Prado', 'Calle 127', 'Pepe Sierra', 'Calle 106', 'Calle 100', 'La Castellana', 'NQS-Calle 75', 'AV. Chile', 'Simon Bolivar', 'Movistar Arena', 'Campin - U. Antonio Nariño', 'U. Nacional', 'A

# **Conclusiones**
Como conclusiones se tiene que:

*   Aunque hayan multiples rutas, dependiendo de la necesidad del usuario cada algoritmo de busqueda puede dar una solucion diferente, la cual puede ser mas optima o no dependiendo de la persona.
*   Los algoritmos de busqueda no informada no son muy diferentes de las busquedas informadas. Los primeros hacen una busqueda a ciegas no tienen mucha informacion mas que el costo por cada conexion entre vertices, mientras que el segundo metodo tiene informacion anticipada, pudiendo tomar mejores decision estre escoger multiples estados en el mundo del problema.
*   El uso de busqueda se puede utilizar en multiples campos, no solo en geolocalizacion y busqueda de una mejor ruta en un mapa dentro de distintas localizaciones





# **Otros usos**

Este proyecto se puede utilizan en diferentes campos de accion, Como en este caso, en geolocalizacion o utilizando otras localizaciones, no necesariamente estaciones, se puede llegar a un resultado con un optimo local o global por cada algoritmo.

Tambien se tiene el campo de juegos, en donde se pueden armar agentes inteligentes en un mundo 2D o 3D y que hagan una busqueda de  un objeto en un mundo multidimensional o bidimensional, haciendo que el juego sea mas realista o interactivo con el jugador, segun sea el caso.

En redes computaciones tiene mucho campo de accion, podrian dotar los equipos de red, como modems, routers, switches, APs para dedicir cual es la mejor ruta dentro de una red y decidir por que camino enviar los paquetes y datos con el menor costo posible.